# Quantum Computing Introduction

*This content is adapted from IBM Quantum Computing textbook. There are lots of other great resources [here](http://qiskit.org/learn/).*

Here you are going to start learning about how a quantum computer (QC) works, and what that even is. A great first question is: why would you want to? 

A Quantum Computer is a 
[different type of computer](https://www.technologyreview.com/2019/01/29/66141/what-is-quantum-computing/), not just better or faster that yours, but one that can solve problems that are virtually impossible for a regular computer. For example, In 2019, 
[Google](https://research.google/pubs/quantum-supremacy-using-a-programmable-superconducting-processor/), used a QC to solve in 3 minutes a math problem that would've taken a regular computer 10,000 years.

A particular terrifying problem that is very hard for a classical computer but potentially easy for a QC is [decrypting your password](https://www.sciencenews.org/article/quantum-computers-break-internet-save). There are positives, too, in that a QC can be used to solve very intractable problems in Biology like [drug discovery](https://www.scientific-computing.com/article/potential-quantum-computing-biological-research).

Job growth in quantum computing has been massive in the past few years, and many big companies are racing to fill a [gap in Quantum skills](https://www.sdxcentral.com/articles/analysis/ibm-microsoft-and-google-race-to-close-quantum-skills-gap/2023/02/) that current graduates don't have. 

IBM has their own set of QCs that anyone can use! In this activity, you will learn IBM's development kit for programming quantum computers, called Qiskit. You'll build and simulate a quantum circuit.

Below are some packages we need to import first.

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

'''
If you haven't used QISKIT before, you'll have to install the libraries. On Mac open a new terminal window, 
or on Windows open the Anaconda Command Prompt. 
Then type the following lines, one at a time, hitting enter after each one. 
They will run for a few seconds, and requires an internet connection. You can tell they are done when you have
an empty command line waiting for input. 
At the end, close and halt this notebook, re-open it, run this cell (shift+enter) to import these 
various packages. If it doesn't produce any error messages, it should have worked.

pip install qiskit        #all of the useful Quantum Circuit tools
pip install qiskit-aer    #for simulating runs
pip install pylatexenc    #for displaying figures in a nice way
''';

## 1. Bits and Binary

Before we jump in the quantum end of the pool, let's remind ourselves how to splash around in the standard, classical end of the computational swimming pool (sorry that analogy got a little slippery). The first thing we need to know about is the idea of bits. All information in a computer (numbers, programs, memes, etc.) are represented by high and low voltages. These might a specific location on a hard drive which as it a high voltage (ed. note: massive simplification). These locations are called *bits*, and when discussing a bit at a high voltage we say is has a value of 1, and a low voltage is 0. 

These are designed to be the world’s simplest alphabet. With only two characters, 0 and 1, we can represent any piece of information, just as I can convey any info with the Latin alphabet.

One example is numbers. You are probably used to representing a number through a string of the ten digits 0, 1, 2, 3, 4, 5, 6, 7, 8, and 9. This is called *base-10*. For example, if I write the number 23, that really means I have 2 "tens" and 3 "ones", or 20+3 = twenty-three. 

When I just have 2 digits, 0 and 1, it similar. This is called *base-2*. In this system, the number 10 means I have 1 "two" and 0 "ones", or just 2. Thus in base-2, "10" means 2. The number 11 in base-2 means I have 1 "two" and 1 "one", or 3. 

Base-2 is also called Binary. We could really unpack this, but for this exercise we won't need to count numbers more than 2, though we will do this kindergarten-level of counting using not-kindergarten-level programming.

***

## 2. Classical Computers 

In a computer you can represent more complex information using longer strings of 1s and 0s. You can see this [this table](https://www.ibm.com/support/knowledgecenter/en/ssw_aix_72/com.ibm.aix.networkcomm/conversion_table.htm) for an idea of how different characters like an ampersand are represented in 1s and 0s. These are basically just definitions all humans have agreed upon.

You can store info in a computer, but the true power is manipulating the data, like adding numbers. Computers manipulate these bits using *gates*. How they are constructed is complicated, but one example might be a NOT gate. This takes in any bit (say it is a 1), does fancy stuff, and then outputs the opposite of that (0). Another gate is an AND gate, which takes in two bits and outputs 1 if the first AND second bit are both 1.

Let's call the first bit *bitA* and the second one *bitB*. We can code up both gates in Python as follows.

In [ ]:
bitA=1  
bitB=0  

output1 = not bitA  #output of a NOT gate
output2 = bitA and bitB  #output of an AND gate

print(output1)
print()
print(output2)

Note that for Python, `False` often means the same as `0`, and `True` the same as `1`. **Run the code above for all possible combinations of *bitA* and *bitB* to make sure the results make sense to you.** Remember each bit can be either 0 or 1.

There are all sorts of other gates, like XOR (just "either or", but the X makes it sound cool), which outputs high if either *bitA* or *bitB* is true but not both. Try it below, again adjusting the input bits.

In [ ]:
bitA=0
bitB=1

output1 = bitA^bitB  #output of an XOR gate, not Python using ^ for an xor

print(output1)

***

### Quick Exercise

1. You have 3 bits, *bitA=0*, *bitB=1*, and *bitC=0*. What do you get if you XOR the result of bitB AND bitC with the NOT of bitA? Code this up in a way such that you can adjust the individual bits and easily get the new result.

In [ ]:
#your code goes here


***

### Adding in Binary

What we want to in this activity is use a computer, first classically and then quantum-ly, to add two numbers. You might know that in binary, the addition tables are:

```
0+0 = 00 (in decimal, this is 0+0=0)
0+1 = 01 (in decimal, this is 0+1=1)
1+0 = 01 (in decimal, this is 1+0=1)
1+1 = 10 (in decimal, this is 1+1=2)
```

We want to build a circuit that can output the right 2-digit number given two input bits, which can be either 1 or 0 (just 1-digit). A circuit that does this is called a *half-adder*. This is one component of being able to add any two numbers of any size, but we dare not dream so big.

### Half adder

To figure out how to program this, look at that table above again. We will call the first input *bit0* and the second input *bit1*. The output, or result, is two digits. Our program isn't going to output the speific number we want (00, 01, etc.), but instead will have two outputs, one for each digit in the sum.

Note first the leftmost digit of the sum is `0` unless both input digits are `1`. This is an AND gate, so the leftmost digit is simply `bit0 AND bit1`. Easy.

The rightmost bit in these answers is `0` if both inputs are the same (`0+0` or `1+1`), and `1` if both inputs are different. Look at the table above and make sure you understand that. This is an XOR gate, where the output is 1 if either bit1 or bit2 is 1, but not both.

Ok, problem solved. Let's code it up, using something you have already learned, a function!

In [ ]:
def half_add(bit0,bit1):
    left_output_bit = bit0 and bit1
    right_output_bit = bit0^bit1
    return(left_output_bit,right_output_bit)


print(half_add(1,0))

***

### Quick Exercise \#3
Copy the code above and run it for all 4 possible input combinations, veryfing it works as intended.

In [ ]:
#your code goes here


***

## 3. Quantum Computers

Enough goofin' around, let's get to quantum computers. So what is a QC? A regular computer's hard drive and processors are made up of different forms of silicon microchips, which are small but still consist of many many atomes. A QC uses single atoms. Each bit in a QC is actually a single atom, called a *qubit*, for *quantum bit*. Atoms, it turns out, spin. And a qubit in the *1* state might be spinning in one direction, and a *qubit* in the *0* state might be spinning in the opposite direction.

Note that there can diffeerent ways of making a QC, such that you are not actually using atoms, but atoms are the easiest way to visualize.

The advantage of a QC is that atoms can be in a weird combination of *1* and *0* states at the same time. Why is this good? On a classical computer, if we wanted to run our half-adder for all four possible input configurations, we have to run that code 4 separate times, changing the input each time. You did this above. For a QC, if we prepare each qubit in a combination of both *1* and *0*, then if set it up right, we only have to run the code once, since all possible input configurations are already present. This might sound small, but with lots of qubits and millions of input configurations, this can save a TREMENDOUS amount of time.

For the moment, we want to learn how to code up our half-adder on a QC. IBM has built a set of actual QCs that anyone can use, and then created some tools to interface with them. These tools are called **Qiskit**, which we will learn now. 

In a circuit, we typically need to do three jobs: encode the input, then do some actual computation, and finally extract an output. Let's start by defining our inputs. IBM's Qiskit has a set of unique Python commands to do this.

In [ ]:
# Create quantum circuit with 2 qubits and 2 classical bits
qc1 = QuantumCircuit(2, 2) 
qc1.draw(initial_state=True,output='mpl')  # returns a drawing of the circuit

The first line above creates 2 quantum bits (q0, or qubit0, and then q1, or qubit1) and 2 classical bits. We'll explain why we need the classical bits in a second. All of them are created in the *0* state. The second line in the code just draws them.

This circuit, which we have called `qc1`, is created by Qiskit using the `QuantumCircuit` command. The first argument of this is the number of qubits in the quantum circuit, and the second is the number of classical bits. The set of qubits is the called the *quantum register* and the set of classical bits the *classical register*.

Play around with this code and see how it changes.

### Encode Input & Measure

The next step is to encode an input. Let's say we want to add 1+0, so qubit0 is 1, and qubit1 is 0. We started above with both 0, se we need to apply a NOT gate to qubit0. For a QC, this NOT gate is denoted by the symbol X.

In [ ]:
qc1 = QuantumCircuit(2,2)
qc1.x(0)
qc1.draw(initial_state=True,output='mpl')

Neat!

We created and changed the state of our qubits, but we need to actually measure them. Notice that the initial state is shown (on the left) but no output state is shown (we can't see that q0 is now 1). To do this, let's measure these 2 qubits.

The extraction of outputs in a quantum circuit is done using an operation called `measure`. Each measurement tells a specific qubit to give an output to a specific classical bit. These classical bits can be read by a normal, lame, computer. How this projection happens in a reality depends on the nature of the real quantum computer, but for now it is just a shortcut way of saying read the qubits. 

In [ ]:
qc1 = QuantumCircuit(2, 2) 
qc1.x(0)
qc1.measure([0,1], [0,1])   #maps qubits 0,1 to classical bits 0,1 respectively.
qc1.draw(initial_state='True',output='mpl')

Notice that little 2 at the bottom is just the number of classical bits, and the litte 1 and 0 are the designation for the classical bit, but not the value of those bits. Try swapping the `0` and `1`'s in the `[0,1], [0,1]` in some way to see what hapens, but when done return to the initial value. 

The code above just sets up a circuit that will initiaze and then measure the qubits. To actually measure, do the following.

In [ ]:
sim_backend = AerSimulator()      # make new simulator object
job_qc1 = sim_backend.run(qc1)    # run the experiment
result_qc1 = job_qc1.result()     # get the results
counts_qc1 = result_qc1.get_counts()    # interpret the results as a "counts" dictionary
plot_histogram(counts_qc1)        #make a histogram of the results

WTH was all the code? The first line says that we are going to run our quantum on a simulator, called Aer. So we aren't running it on an actual computer, but simulating it, which is always good practice before asking IBM to use their real quantum computer. This simulator is called the *backend* of the circuit. 

I won't unpack the other lines, but it is just figuring out how display the results in the histogram shown above.

Aer actually runs the circuit 1000 times, and measures the result each time. The reason for doing and showing the result as a histogram is because quantum computers may have some randomness in their results. Since we aren’t using a real quantum computer yet, we get just the `01` every time, but a real QC might get a wrong answer, like `11` a few times out of a 1000.

### Quantum Gates - CNOT

Next we need to do computation on the qubits, like the AND and XOR gates we used earlier. 

For a QC, gates are not quite the same. One common one is called a Controlled-Not (or CNOT). This is shown below.

<img src="images/cnot_xor.svg">

This is CNOT applied to a pair of qubits. One acts as the *control qubit* (the one with the little dot). The other acts as the *target qubit* (with the big circle that has a ```+``` inside it).

There are multiple ways to explain the effect of the CNOT. One is to say the target bit is flipped (NOT-ed) if the control bit is 1, and the target bit stays the same if the control bit is 0. The code for implementing this is as follows. The Qiskit name for CNOT is ```cx```.

In [ ]:
qc_cnot = QuantumCircuit(2)
qc_cnot.cx(0,1)
qc_cnot.draw(initial_state=True,output='mpl')

It turns out a CNOT is related to an XOR, as we will see in a bit.

***

### Quick Exercise \#2
Before we go too far, let's stop and stretch our minds. Make a Quantum circuit with 2 qubits in the quantum register and 1 classical bit in the classical register. Call this circuit `qc_test`. Invert qubit0 so it is in the 1 state, and then make a CNOT gate with qubit0 as the control and qubit1 as the target. Set it up the measure qubit1 using the classical bit (called bit 0), and then draw the circuit. Don't actually run the simulator to display the results, just draw the circuit.

In [ ]:
#put your code here. It should be 5 lines.


***

### Quantum Gates - Toffoli

We are now actually halfway to a fully working half adder (quarter adder?). Another common quantum gate is a *Toffoli Gate*. This takes three qubits, and if the first 2 quibts are 1 then it flips the third. It sometimes called a controlled-controlled-not, because there are two controls. In Qiskit it is called ```ccx```. It looks like this:

In [ ]:
qc_toff = QuantumCircuit(3)
qc_toff.ccx(0,1,2)
qc_toff.draw(initial_state=True,output='mpl')

## 4. Quantum Half-Adder

We now have all we need to add 2 numbers using a quantum computer. Recall we want it to be able to add two 1-digit binary numbers:

```
0+0 = 00
0+1 = 01
1+0 = 01
1+1 = 10
```

We will have two input qubits: qubit0 and qubit1. We need two output qubits, one for the rightmost digit of the output and one for the leftmost digit. We will call these qubit3 and qubit2, respectively. When we did the classical half-adder, we said the leftmost digit was determined from and AND operation. For our qubits then, we then need `qubit3 = qubit0 AND qubit1`, where the AND is now a Toffoli Gate. 

For the rightmost output, stored on `qubit2`, look closely and see if this is by default 0, this digit will flip once for each of the inputs that is 1. For example, in the second row, `1+0=01`, think if the rightmost digit as starting at `0` but flipping once because the first input is 1. For `1+1=10`, it starts at `0`, then flips twice, so it flips once `0->1`, and then again `1->0`. To implement this in our QC, we then need to 2 CNOT gates.

You are going to code this up yourself! Worry not, I will walk you through it, and we may go through it as a class.

***

### Half-Adder Exercise 

Code up below the entire Quantum Half-Adder. Let's start with adding `1` and `1`. The steps you should take are as follows. Do these one by one, entering the code in the cell below.

- Create a Quantum Circuit with 4 qubits and 2 in the classical bits. Call this `qc_ha`.
- Invert both qubits 0 and 1 so they are in the 1 state initially.
- Create a Toffoli gate with qubit0 and qubit1 as the controls, and qubit3 as the target.
- Create a CNOT with qubit0 as the control and qubit2 as the target, and then another CNOT with qubit1 as the control and qubit2 as the target.
- Setup the circuit to measure qubit2 using classical bit0, and qubit3 using classical bit1.
- Draw your glorious circuit.

In [ ]:
#put your code here


That was drawing, and make sure it seems correct to you. Now copy the measurement code from the previous example, using the Aer Simulator, to measure your `qc_ha` circuit and display the resulting histogram. Do you get what you expect?

***

## 5. Conclusions

This is just a a taste of how to program a QC. Learning about the real advantage of a QC takes more math and physics, and obviously you don't need a next-gen computer to add 1 and 1, but there are many many other gates and algorithms for solving complex problems. IBM's page on [Quantum Learning](http://qiskit.org/learn/) has way more activities and information, including a certificate program.

 If you want to explore and learn more, that IBM page is a good place to start, but it get's very mathy very quickly. You can start with some of the following problems:
 
 1. Learn what a full-adder is, what role is plays in adding two arbitrary numbers, and implement that using Qiskit.
 2. Learn what a Hadamard Gate is and how to implement it, and see what happens when initialize your bits using it.
 3. There are other standard classical gates, like *OR* and *NOR* (neither or), and *XNOR* (exclusive or). Figure out what each of these do classically, and how implement them with Qiskit. Code them up with Qiskit and verify they give the output you expect.
 4. Quantum Computers need to be *reversible*, meaning that if you take the output of any quantum circuit, and run it backwards through the same computer, you recover the input qubits. This is not true for regular computers, in that if the calculator on my computer gives me a *6*, I can't run it backwards and figure out if that came from 2 times 3 or 6 times 1. Come up with some random set of quantum gates and input qubits, run it, and then figure out how to run it backwards and verify your circuit is reversible.
